# Upload SVD inputs and outputs to database
Natalia Vélez, November 2020

In [1]:
import pymongo
import os
import bson
import json
import glob
import pickle
import gridfs
import pandas as pd
import numpy as np
from os.path import join as opj
from tqdm import notebook

Connect to database:

In [2]:
keyfile = 'credentials.key'
creds = open(keyfile, "r").read().splitlines()
myclient = pymongo.MongoClient('134.76.24.75', username=creds[0], password=creds[1], authSource='ohol') 
db = myclient.ohol

print(db)
print(db.list_collection_names())

Database(MongoClient(host=['134.76.24.75:27017'], document_class=dict, tz_aware=False, connect=True, authsource='ohol'), 'ohol')
['families', 'test', 'lifelogs', 'fs.files', 'svd', 'fs.chunks']


## Jobmatrix

List job files:

In [3]:
job_files = glob.glob(r'../3_technology/outputs/jobmatrix/*[0123456789].txt')
label_files = glob.glob(r'../3_technology/outputs/jobmatrix/*labels.txt')

job_files.sort()
print(*job_files[:10], sep='\n')
print(*label_files[:10], sep='\n')

../3_technology/outputs/jobmatrix/jobmatrix_release-284_start-1573895672.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-284_start-1573982073.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-284_start-1574068473.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-285_start-1574102503.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-287_start-1574151678.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-287_start-1574238079.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-287_start-1574324479.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-287_start-1574410879.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-287_start-1574497279.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-289_start-1574552311.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-344_start-1593247277_labels.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-346_start-1597023293_labels.txt
../3_technology/outputs/jobmatrix/jobmatrix_release-336_start-

Process files:

In [5]:
job_dict = {}

for f in notebook.tqdm(job_files):
    label_f = f.replace('.txt', '_labels.txt')

    labels = np.loadtxt(label_f).astype(int)
    labels = list(labels)

    mtx = np.loadtxt(f)
    mtx = np.split(mtx, mtx.shape[0])
    mtx = [np.squeeze(l) for l in mtx]

    for avatar,items in zip(labels,mtx):
        if avatar in job_dict:
            print('Merging entries for avatar: %i' % avatar)
            job_dict[avatar] = job_dict[avatar] + items
        else:
            job_dict[avatar] = items

Merging entries for avatar: 2281355
Merging entries for avatar: 2281357
Merging entries for avatar: 2281363
Merging entries for avatar: 2281365
Merging entries for avatar: 2281367
Merging entries for avatar: 2281382
Merging entries for avatar: 2281395
Merging entries for avatar: 2281415
Merging entries for avatar: 2281432
Merging entries for avatar: 2281434
Merging entries for avatar: 2281435
Merging entries for avatar: 2281436
Merging entries for avatar: 2281440
Merging entries for avatar: 2281442
Merging entries for avatar: 2281443
Merging entries for avatar: 2281445
Merging entries for avatar: 2281449
Merging entries for avatar: 2281450
Merging entries for avatar: 2281453
Merging entries for avatar: 2281454
Merging entries for avatar: 2281459
Merging entries for avatar: 2281460
Merging entries for avatar: 2281461
Merging entries for avatar: 2281463
Merging entries for avatar: 2281464
Merging entries for avatar: 2281465
Merging entries for avatar: 2281466
Merging entries for avatar: 

Merging entries for avatar: 2319870
Merging entries for avatar: 2319879
Merging entries for avatar: 2319880
Merging entries for avatar: 2319884
Merging entries for avatar: 2319889
Merging entries for avatar: 2319897
Merging entries for avatar: 2319898
Merging entries for avatar: 2319903
Merging entries for avatar: 2319905
Merging entries for avatar: 2319912
Merging entries for avatar: 2319914
Merging entries for avatar: 2319917
Merging entries for avatar: 2319919
Merging entries for avatar: 2319921
Merging entries for avatar: 2319926
Merging entries for avatar: 2319927
Merging entries for avatar: 2319928
Merging entries for avatar: 2319933
Merging entries for avatar: 2319939
Merging entries for avatar: 2319943
Merging entries for avatar: 2319944
Merging entries for avatar: 2319945
Merging entries for avatar: 2319947
Merging entries for avatar: 2319948
Merging entries for avatar: 2319949
Merging entries for avatar: 2319950
Merging entries for avatar: 2319952
Merging entries for avatar: 

Merging entries for avatar: 2342907
Merging entries for avatar: 2342908
Merging entries for avatar: 2342915
Merging entries for avatar: 2342916
Merging entries for avatar: 2342923
Merging entries for avatar: 2342924
Merging entries for avatar: 2342931
Merging entries for avatar: 2342934
Merging entries for avatar: 2342936
Merging entries for avatar: 2342938
Merging entries for avatar: 2342949
Merging entries for avatar: 2342955
Merging entries for avatar: 2342960
Merging entries for avatar: 2342978
Merging entries for avatar: 2342979
Merging entries for avatar: 2342989
Merging entries for avatar: 2342994
Merging entries for avatar: 2342995
Merging entries for avatar: 2342999
Merging entries for avatar: 2343000
Merging entries for avatar: 2343005
Merging entries for avatar: 2343008
Merging entries for avatar: 2343014
Merging entries for avatar: 2343015
Merging entries for avatar: 2343017
Merging entries for avatar: 2343020
Merging entries for avatar: 2343023
Merging entries for avatar: 

Merging entries for avatar: 2365984
Merging entries for avatar: 2365987
Merging entries for avatar: 2365992
Merging entries for avatar: 2365993
Merging entries for avatar: 2365994
Merging entries for avatar: 2365997
Merging entries for avatar: 2365998
Merging entries for avatar: 2366000
Merging entries for avatar: 2366001
Merging entries for avatar: 2366004
Merging entries for avatar: 2366005
Merging entries for avatar: 2366007
Merging entries for avatar: 2366011
Merging entries for avatar: 2366021
Merging entries for avatar: 2366022
Merging entries for avatar: 2366023
Merging entries for avatar: 2366024
Merging entries for avatar: 2366025
Merging entries for avatar: 2366027
Merging entries for avatar: 2366028
Merging entries for avatar: 2366029
Merging entries for avatar: 2366031
Merging entries for avatar: 2366033
Merging entries for avatar: 2366034
Merging entries for avatar: 2366036
Merging entries for avatar: 2366038
Merging entries for avatar: 2366040
Merging entries for avatar: 

Merging entries for avatar: 2390178
Merging entries for avatar: 2390179
Merging entries for avatar: 2390180
Merging entries for avatar: 2390182
Merging entries for avatar: 2390184
Merging entries for avatar: 2390190
Merging entries for avatar: 2390191
Merging entries for avatar: 2390199
Merging entries for avatar: 2390202
Merging entries for avatar: 2390207
Merging entries for avatar: 2390208
Merging entries for avatar: 2390210
Merging entries for avatar: 2390212
Merging entries for avatar: 2390214
Merging entries for avatar: 2390219
Merging entries for avatar: 2390226
Merging entries for avatar: 2390246
Merging entries for avatar: 2390254
Merging entries for avatar: 2390257
Merging entries for avatar: 2390258
Merging entries for avatar: 2390259
Merging entries for avatar: 2390266
Merging entries for avatar: 2390267
Merging entries for avatar: 2390269
Merging entries for avatar: 2390271
Merging entries for avatar: 2390274
Merging entries for avatar: 2390282
Merging entries for avatar: 

...

Merging entries for avatar: 3228921
Merging entries for avatar: 3228922
Merging entries for avatar: 3228923
Merging entries for avatar: 3228926
Merging entries for avatar: 3228927
Merging entries for avatar: 3228928
Merging entries for avatar: 3228930
Merging entries for avatar: 3228931
Merging entries for avatar: 3228932
Merging entries for avatar: 3228933
Merging entries for avatar: 3228935
Merging entries for avatar: 3228936
Merging entries for avatar: 3228937
Merging entries for avatar: 3228938
Merging entries for avatar: 3228939
Merging entries for avatar: 3228941
Merging entries for avatar: 3228944
Merging entries for avatar: 3228945
Merging entries for avatar: 3228949
Merging entries for avatar: 3228950
Merging entries for avatar: 3228953
Merging entries for avatar: 3228954
Merging entries for avatar: 3228969
Merging entries for avatar: 3228972
Merging entries for avatar: 3228976
Merging entries for avatar: 3228977
Merging entries for avatar: 3228978
Merging entries for avatar: 

Merging entries for avatar: 3242999
Merging entries for avatar: 3243000
Merging entries for avatar: 3243005
Merging entries for avatar: 3243007
Merging entries for avatar: 3243019
Merging entries for avatar: 3243025
Merging entries for avatar: 3243033
Merging entries for avatar: 3243034
Merging entries for avatar: 3243045
Merging entries for avatar: 3243049
Merging entries for avatar: 3243054
Merging entries for avatar: 3243058
Merging entries for avatar: 3243060
Merging entries for avatar: 3243064
Merging entries for avatar: 3243065
Merging entries for avatar: 3243066
Merging entries for avatar: 3243068
Merging entries for avatar: 3243069
Merging entries for avatar: 3243070
Merging entries for avatar: 3243072
Merging entries for avatar: 3243073
Merging entries for avatar: 3243074
Merging entries for avatar: 3243075
Merging entries for avatar: 3243076
Merging entries for avatar: 3243077
Merging entries for avatar: 3243079
Merging entries for avatar: 3243080
Merging entries for avatar: 

Merging entries for avatar: 3258800
Merging entries for avatar: 3258804
Merging entries for avatar: 3258806
Merging entries for avatar: 3258807
Merging entries for avatar: 3258808
Merging entries for avatar: 3258809
Merging entries for avatar: 3258813
Merging entries for avatar: 3258823
Merging entries for avatar: 3258824
Merging entries for avatar: 3258825
Merging entries for avatar: 3258827
Merging entries for avatar: 3258829
Merging entries for avatar: 3258830
Merging entries for avatar: 3258831
Merging entries for avatar: 3258833
Merging entries for avatar: 3258834
Merging entries for avatar: 3258836
Merging entries for avatar: 3258837
Merging entries for avatar: 3258838
Merging entries for avatar: 3258841
Merging entries for avatar: 3258842
Merging entries for avatar: 3258844
Merging entries for avatar: 3258849
Merging entries for avatar: 3258853
Merging entries for avatar: 3258858
Merging entries for avatar: 3261030
Merging entries for avatar: 3261032
Merging entries for avatar: 

Merging entries for avatar: 3278282
Merging entries for avatar: 3278283
Merging entries for avatar: 3278285
Merging entries for avatar: 3278288
Merging entries for avatar: 3278290
Merging entries for avatar: 3278292
Merging entries for avatar: 3278295
Merging entries for avatar: 3278304
Merging entries for avatar: 3278305
Merging entries for avatar: 3278306
Merging entries for avatar: 3278308
Merging entries for avatar: 3278309
Merging entries for avatar: 3278311
Merging entries for avatar: 3278312
Merging entries for avatar: 3278314
Merging entries for avatar: 3278318
Merging entries for avatar: 3278323
Merging entries for avatar: 3278324
Merging entries for avatar: 3278325
Merging entries for avatar: 3278326
Merging entries for avatar: 3278327
Merging entries for avatar: 3278328
Merging entries for avatar: 3278332
Merging entries for avatar: 3278338
Merging entries for avatar: 3278339
Merging entries for avatar: 3278340
Merging entries for avatar: 3280630
Merging entries for avatar: 

Merging entries for avatar: 3294887
Merging entries for avatar: 3294891
Merging entries for avatar: 3294893
Merging entries for avatar: 3294898
Merging entries for avatar: 3294899
Merging entries for avatar: 3294902
Merging entries for avatar: 3294908
Merging entries for avatar: 3294910
Merging entries for avatar: 3294912
Merging entries for avatar: 3294913
Merging entries for avatar: 3294916
Merging entries for avatar: 3294919
Merging entries for avatar: 3294920
Merging entries for avatar: 3294921
Merging entries for avatar: 3294927
Merging entries for avatar: 3294928
Merging entries for avatar: 3294932
Merging entries for avatar: 3294934
Merging entries for avatar: 3294942
Merging entries for avatar: 3294948
Merging entries for avatar: 3294949
Merging entries for avatar: 3294950
Merging entries for avatar: 3294951
Merging entries for avatar: 3294952
Merging entries for avatar: 3294953
Merging entries for avatar: 3294954
Merging entries for avatar: 3294955
Merging entries for avatar: 

Convert to list:

In [ ]:
job_records = [{'avatar': k, 'items': list(v)} for k,v in job_dict.items()]

Save to collection:

In [ ]:
print('Saving %i entries' % len(job_dict.keys()))
job_col = db.jobmatrix
job_col.insert_many(job_records)

## SVD

Load SVD data:

In [ ]:
U = np.loadtxt('../3_technology/outputs/svd/U_hat.txt')
s = np.loadtxt('../3_technology/outputs/svd/s_hat.txt')
V = np.loadtxt('../3_technology/outputs/svd/Vh_hat.txt')
avatars = np.loadtxt('../3_technology/outputs/svd/input_playerIDs.txt')
items = np.loadtxt('../3_technology/outputs/activity_features.txt')

Record inputs

Assemble record:

In [ ]:
svd_record = {}

# Serialize 1d entries
svd_record['s'] = list(s)

avatar_out = avatars.astype(int)
svd_record['avatars'] = list(avatars)

#item_out = items.astype(int)
svd_record['items'] = list(items)

# Pickle 2d entries
U_bin = bson.binary.Binary(pickle.dumps(U, protocol=2), subtype=128)
V_bin = bson.binary.Binary(pickle.dumps(U, protocol=2), subtype=128)

Save large files using GridFS instance:

In [ ]:
fs = gridfs.GridFS(db)
U_id = fs.put(U_bin, filename='svd_U')
V_id = fs.put(V_bin, filename='svd_V')
print(U_id)
print(V_id)

In [ ]:
print(db.list_collection_names())

In [ ]:
db.fs.chunks

Save everything else to a collection:

In [ ]:
svd_col = db.svd
svd_col.drop()

In [ ]:
svd_col = db.svd
svd_col.insert_one(svd_record)